In [1]:
%config Completer.use_jedi = False

In [2]:
import xpypact as xp

In [3]:
xp.__version__

'0.1.12a0'

In [4]:
from pathlib import Path

In [5]:
root_dir = Path("~", "dev", "xpypact").expanduser()

In [6]:
json_path = root_dir / "wrk/Alloy718-Co04-104_2_1_1.json"
assert json_path.exists()

In [7]:
from xpypact.Inventory import Inventory, from_json
import xpypact.data_arrays as da

In [8]:
inventory = from_json(json_path)

In [9]:
ds = da.create_dataset(inventory)

In [10]:
ds

<xarray.Dataset>
Dimensions:                 (time_step_number: 65, nuclide: 215,
                             gamma_boundaries: 25, timestamp: 1)
Coordinates:
  * time_step_number        (time_step_number) int64 1 2 3 4 5 ... 62 63 64 65
  * nuclide                 (nuclide) object MultiIndex
  * element                 (nuclide) object 'Al' 'Al' 'Al' ... 'Zr' 'Zr' 'Zr'
  * mass_number             (nuclide) int64 26 27 28 29 30 40 ... 93 94 95 96 97
  * state                   (nuclide) object '' '' '' '' '' ... '' '' '' '' ''
  * gamma_boundaries        (gamma_boundaries) float64 1e-11 0.01 ... 14.0 20.0
    elapsed_time            (time_step_number) float64 0.0 6.312e+07 ... 3.2e+10
    zai                     (nuclide) int64 130260 130270 ... 400960 400970
  * timestamp               (timestamp) datetime64[ns] 2022-02-21T01:52:47
Data variables: (12/32)
    irradiation_time        (time_step_number) float64 0.0 ... 4.207e+08
    cooling_time            (time_step_number) float64 0.0 0.0 ... 3.158e+10
    duration                (time_step_number) float64 0.0 ... 2.84e+10
    flux                    (time_step_number) float64 0.0 2.445e+10 ... 0.0 0.0
    total_atoms             (time_step_number) float64 1.045e+22 ... 1.045e+22
    total_activity          (time_step_number) float64 0.01099 ... 3.152e+04
    ...                      ...
    nuclide_beta_heat       (time_step_number, nuclide) float64 nan 0.0 ... nan
    nuclide_gamma_heat      (time_step_number, nuclide) float64 nan 0.0 ... nan
    nuclide_dose            (time_step_number, nuclide) float64 nan 0.0 ... nan
    nuclide_ingestion       (time_step_number, nuclide) float64 nan 0.0 ... nan
    nuclide_inhalation      (time_step_number, nuclide) float64 nan 0.0 ... nan
    gamma                   (time_step_number, gamma_boundaries) float64 0.0 ...
Attributes:
    run_name:            * Material Alloy718-Co04, fluxes 104_2_1_1
    flux_name:           90-degrees bend
    dose_rate_type:      Point source
    dose_rate_distance:  1.0

In [12]:
import duckdb as db

In [13]:
db_path = root_dir / "wrk/try-duckdb.duckdb"
db_path.parent.mkdir(parents=True, exist_ok=True)
if db_path.exists():
    db_path.unlink()

In [14]:
con = db.connect(str(db_path))

In [15]:
material_id = 100
case_id = 3

In [16]:
ds =ds.expand_dims(dim={"material_id": [material_id], "case_id": [case_id]})
ds

<xarray.Dataset>
Dimensions:                 (material_id: 1, case_id: 1, time_step_number: 65,
                             nuclide: 215, gamma_boundaries: 25, timestamp: 1)
Coordinates:
  * material_id             (material_id) int64 100
  * case_id                 (case_id) int64 3
  * time_step_number        (time_step_number) int64 1 2 3 4 5 ... 62 63 64 65
  * nuclide                 (nuclide) object MultiIndex
  * element                 (nuclide) object 'Al' 'Al' 'Al' ... 'Zr' 'Zr' 'Zr'
  * mass_number             (nuclide) int64 26 27 28 29 30 40 ... 93 94 95 96 97
  * state                   (nuclide) object '' '' '' '' '' ... '' '' '' '' ''
  * gamma_boundaries        (gamma_boundaries) float64 1e-11 0.01 ... 14.0 20.0
    elapsed_time            (time_step_number) float64 0.0 6.312e+07 ... 3.2e+10
    zai                     (nuclide) int64 130260 130270 ... 400960 400970
  * timestamp               (timestamp) datetime64[ns] 2022-02-21T01:52:47
Data variables: (12/32)
    irradiation_time        (material_id, case_id, time_step_number) float64 ...
    cooling_time            (material_id, case_id, time_step_number) float64 ...
    duration                (material_id, case_id, time_step_number) float64 ...
    flux                    (material_id, case_id, time_step_number) float64 ...
    total_atoms             (material_id, case_id, time_step_number) float64 ...
    total_activity          (material_id, case_id, time_step_number) float64 ...
    ...                      ...
    nuclide_beta_heat       (material_id, case_id, time_step_number, nuclide) float64 ...
    nuclide_gamma_heat      (material_id, case_id, time_step_number, nuclide) float64 ...
    nuclide_dose            (material_id, case_id, time_step_number, nuclide) float64 ...
    nuclide_ingestion       (material_id, case_id, time_step_number, nuclide) float64 ...
    nuclide_inhalation      (material_id, case_id, time_step_number, nuclide) float64 ...
    gamma                   (material_id, case_id, time_step_number, gamma_boundaries) float64 ...
Attributes:
    run_name:            * Material Alloy718-Co04, fluxes 104_2_1_1
    flux_name:           90-degrees bend
    dose_rate_type:      Point source
    dose_rate_distance:  1.0

How to transform xarray to pandas saving indexes as columns

In [20]:
ds.irradiation_time.stack({"idx":["material_id", "case_id", "time_step_number"]}).to_pandas().reset_index()

,material_id,case_id,time_step_number,0
0,100,3,1,0.0
1,100,3,2,63115200.0
2,100,3,3,378691200.0
3,100,3,4,378691200.0
4,100,3,5,420662808.0
...,...,...,...,...
60,100,3,61,420670808.0
61,100,3,62,420670808.0
62,100,3,63,420670808.0
63,100,3,64,420670808.0


In [21]:
def drop_tables(con):
    con.execute("drop table if exists timestep_gamma")
    con.execute("drop table if exists timestep_nuclide")
    con.execute("drop table if exists timestep")
    con.execute("drop table if exists nuclide")
    con.execute("drop table if exists rundata")

In [22]:
def create_tables(con):
    sql = """ 
    CREATE TABLE IF NOT EXISTS rundata (
        material_id uinteger not null,
        case_id uinteger not null,
        timestamp timestamp not null,
        run_name varchar not null,
        flux_name varchar NOT NULL,
        dose_rate_type varchar NOT NULL,
        dose_rate_distance real NOT NULL,
        primary key(material_id, case_id)
    );

    CREATE TABLE IF NOT EXISTS timestep(
        material_id  uinteger not null,
        case_id      uinteger not null,
        time_step_number uinteger not null,
        elapsed_time float4 not null,
        irradiation_time float4 not null,
        cooling_time float4 not null,
        duration float4 not null,
        flux float4 not null,
        total_atoms float4 not null,
        total_activity float4 not null,
        total_alpha_activity float4 not null,
        total_beta_activity float4 not null,
        total_gamma_activity float4 not null,
        total_mass float4 not null,
        total_heat float4 not null,
        total_alpha_heat float4 null,
        total_beta_heat float4 not null,
        total_gamma_heat float4 not null,
        total_ingest1ion_dose float4 not null,
        total_inhalation_dose float4 not null,
        total_dose_rate float4 not null,
        primary key(material_id, case_id, time_step_number),
        foreign key(material_id, case_id) references rundata(material_id, case_id) 
    );

    
    CREATE TABLE IF NOT EXISTS nuclide(
        element varchar(2) not null,
        mass_number usmallint not null check(0 < mass_number),
        state varchar(1) not null,
        zai integer not null check(10010 <= zai) unique,
        half_life float4 not null check(0 <= half_life),
        primary key(element, mass_number, state)
    );
    
    CREATE TABLE IF NOT EXISTS timestep_nuclide(
        material_id  uinteger not null,
        case_id      uinteger not null,
        time_step_number uinteger not null,
        element varchar(2) not null,
        mass_number usmallint not null,
        
        state varchar(1) not null,
        atoms float4 not null,
        grams float4 not null,
        activity float4 not null,
        alpha_activity float4 not null,
        
        beta_activity float4 not null,
        gamma_activity float4 not null,
        heat float4 not null,
        alpha_heat float4 not null,
        beta_heat float4 not null,
        
        gamma_heat float4 not null,
        dose float4 not null,
        ingestion float4 not null,
        inhalation float4 not null,
        
        primary key(material_id, case_id, time_step_number, element, mass_number, state),
        foreign key(material_id, case_id, time_step_number) references timestep(material_id, case_id, time_step_number),
        foreign key(element, mass_number, state) references nuclide(element, mass_number, state) 
    );

    CREATE TABLE IF NOT EXISTS timestep_gamma(
        material_id  uinteger not null,
        case_id      uinteger not null,
        time_step_number uinteger not null,
        boundary real not null check(0 <= boundary),
        intensity real not null, 
        primary key(material_id, case_id, time_step_number, boundary),
        foreign key(material_id, case_id, time_step_number) references timestep(material_id, case_id, time_step_number),
    );
    """
    con.execute(sql)

In [23]:
drop_tables(con)
create_tables(con)

In [24]:
def save_rundata(con, ds):
    sql = """
        INSERT INTO rundata values(?, ?, ?, ?, ?, ?, ?)
    """
    con.execute(sql, (    
        ds.material_id.item(),
        ds.case_id.item(),
        ds.timestamp.dt.strftime("%Y-%m-%d %H:%M:%S").item(),
        ds.attrs["run_name"],
        ds.attrs["flux_name"],
        ds.attrs["dose_rate_type"],
        ds.attrs["dose_rate_distance"]
    ))
    con.commit()

In [25]:
save_rundata(con, ds)

In [26]:
con.execute("select * from rundata").df()

,material_id,case_id,timestamp,run_name,flux_name,dose_rate_type,dose_rate_distance
0,100,3,2022-02-21 01:52:47,"* Material Alloy718-Co04, fluxes 104_2_1_1",90-degrees bend,Point source,1.0


In [27]:
def save_timesteps(con, ds):
    timesteps_df = ds[
        [
            "material_id",
            "case_id",
            "time_step_number",
            "elapsed_time",
            "irradiation_time",
            "cooling_time",
            "duration",
            "flux",
            "total_atoms",
            "total_activity",
            "total_alpha_activity",
            "total_beta_activity",
            "total_gamma_activity",
            "total_mass",
            "total_heat",
            "total_alpha_heat",
            "total_beta_heat",
            "total_gamma_heat",
            "total_ingest1ion_dose",
            "total_inhalation_dose",
            "total_dose_rate",
        ]
    ].to_dataframe().reset_index()
    sql = "insert into timestep select * from timesteps_df"
    con.execute(sql)
    con.commit()

In [28]:
save_timesteps(con, ds)

In [29]:
con.execute("select * from timestep").df()

,material_id,case_id,time_step_number,elapsed_time,irradiation_time,cooling_time,duration,flux,total_atoms,total_activity,...,total_beta_activity,total_gamma_activity,total_mass,total_heat,total_alpha_heat,total_beta_heat,total_gamma_heat,total_ingest1ion_dose,total_inhalation_dose,total_dose_rate
0,100,3,1,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.045481e+22,1.099260e-02,...,1.099260e-02,0.000000e+00,0.001,1.994928e-19,0.000000e+00,1.994919e-19,8.784190e-25,5.288239e-12,9.131853e-12,7.484415e-22
1,100,3,2,6.311520e+07,63115200.0,0.000000e+00,6.311520e+07,2.445200e+10,1.045481e+22,2.268685e+07,...,1.476443e+07,7.922416e+06,0.001,2.522555e-09,1.316125e-30,4.443058e-10,2.078249e-09,1.225884e-02,7.370374e-02,1.774444e-06
2,100,3,3,3.786912e+08,378691200.0,0.000000e+00,3.155760e+08,1.882800e+11,1.045484e+22,2.005898e+08,...,1.391144e+08,6.147540e+07,0.001,2.601587e-08,6.436155e-28,3.691712e-09,2.232416e-08,1.498245e-01,1.057001e+00,1.854623e-05
3,100,3,4,3.997401e+08,378691200.0,2.104892e+07,2.104892e+07,0.000000e+00,1.045484e+22,5.064473e+07,...,5.005196e+07,5.927796e+05,0.001,1.072963e-08,6.445708e-28,5.938637e-10,1.013577e-08,8.618334e-02,7.240407e-01,7.931148e-06
4,100,3,5,4.417117e+08,420662816.0,2.104892e+07,4.197161e+07,3.790000e+11,1.045484e+22,3.644599e+08,...,2.411954e+08,1.232645e+08,0.001,4.388052e-08,9.368928e-28,7.024670e-09,3.685585e-08,2.313087e-01,1.522821e+00,3.102856e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,100,3,61,7.573741e+08,420670816.0,3.367033e+08,2.209032e+08,0.000000e+00,1.045484e+22,1.021210e+07,...,9.725966e+06,4.861311e+05,0.001,3.030422e-09,9.414649e-28,1.199240e-10,2.910498e-09,2.524398e-02,2.288691e-01,2.231632e-06
61,100,3,62,1.388526e+09,420670816.0,9.678553e+08,6.311520e+08,0.000000e+00,1.045484e+22,2.140445e+06,...,1.934185e+06,2.062603e+05,0.001,2.260388e-10,9.414649e-28,1.325084e-11,2.127880e-10,2.034489e-03,1.899889e-02,1.650785e-07
62,100,3,63,2.019678e+09,420670816.0,1.599007e+09,6.311520e+08,0.000000e+00,1.045484e+22,1.354644e+06,...,1.266816e+06,8.782794e+04,0.001,2.303827e-11,9.414649e-28,4.691881e-12,1.834639e-11,3.442937e-04,3.528395e-03,1.529847e-08
63,100,3,64,3.597558e+09,420670816.0,3.176887e+09,1.577880e+09,0.000000e+00,1.045484e+22,8.909484e+05,...,8.798868e+05,1.106160e+04,0.001,6.016502e-12,9.414649e-28,2.779170e-12,3.237332e-12,1.548528e-04,1.769614e-03,2.999845e-09


In [55]:
columns_all = ["element", "mass_number", "state", "zai", "nuclide_half_life"]
columns = ["zai", "nuclide_half_life"]
ds[columns].to_dataframe()[columns].reset_index()[columns_all]
# .unstack("nuclide") 
# .stack({"idx": ["material_id", "case_id", "element", "mass_number", "state"]}) \
# .to_dataframe()

# \
#     .reset_index(["material_id", "case_id"]) \
#     .to_dataframe() \
#     .reset_index(
#         [
#             "material_id", "case_id"
#         ],
#         drop=True
#     ) \
#     .reset_index(drop=True)

,element,mass_number,state,zai,nuclide_half_life
0,Al,26,,130260,2.262630e+13
1,Al,27,,130270,0.000000e+00
2,Al,28,,130280,1.344600e+02
3,Al,29,,130290,3.936000e+02
4,Al,30,,130300,3.650000e+00
...,...,...,...,...,...
210,Zr,93,,400930,4.828210e+13
211,Zr,94,,400940,1.893460e+23
212,Zr,95,,400950,5.532360e+06
213,Zr,96,,400960,1.230750e+27


In [56]:
def save_nuclides(con, ds):
    columns_all = ["element", "mass_number", "state", "zai", "nuclide_half_life"]
    columns = ["zai", "nuclide_half_life"]
    nuclides_df = ds[columns].to_dataframe()[columns].reset_index()[columns_all]
    sql = "insert or ignore into nuclide select * from nuclides_df"
    con.execute(sql)
    con.commit()

In [57]:
save_nuclides(con, ds)

In [58]:
con.execute("select * from nuclide").df()

,element,mass_number,state,zai,half_life
0,Al,26,,130260,2.262630e+13
1,Al,27,,130270,0.000000e+00
2,Al,28,,130280,1.344600e+02
3,Al,29,,130290,3.936000e+02
4,Al,30,,130300,3.650000e+00
...,...,...,...,...,...
210,Zr,93,,400930,4.828210e+13
211,Zr,94,,400940,1.893460e+23
212,Zr,95,,400950,5.532360e+06
213,Zr,96,,400960,1.230750e+27


In [91]:
columns = [
    # "material_id",
    # "case_id",
    # "time_step_number",
    # "element",
    # "mass_number",
    # "state",
    "nuclide_atoms",
    "nuclide_grams",
    "nuclide_activity",
    "nuclide_alpha_activity",
    "nuclide_beta_activity",
    "nuclide_gamma_activity",
    "nuclide_heat",
    "nuclide_alpha_heat",
    "nuclide_beta_heat",
    "nuclide_gamma_heat",
    "nuclide_dose",
    "nuclide_ingestion",
    "nuclide_inhalation",
]
columns_all = [
    "material_id",
    "case_id",
    "time_step_number",
    "element",
    "mass_number",
    "state",
] + columns

ds[columns].stack(idx=("material_id", "case_id", "time_step_number")).to_dataframe()[columns].reset_index()[columns_all].fillna(0.0)
# ds[columns].reset_index(["material_id", "case_id"])

# ds[columns].coords.to_dataset()
# .reset_index(["elapsed_time", "zai"])
# .reset_coords()
# .drop_indexes(["material_id", "case_id", "time_step_number", "nuclide"])
# .reset_index(["material_id", "case_id", "time_step_number", "nuclide"])
# .to_dataframe().reset_index(
#     [
#         "material_id", "case_id", "time_step_number",
#     ]
# ).reset_index(drop=True)[columns].fillna(0.0)


,material_id,case_id,time_step_number,element,mass_number,state,nuclide_atoms,nuclide_grams,nuclide_activity,nuclide_alpha_activity,nuclide_beta_activity,nuclide_gamma_activity,nuclide_heat,nuclide_alpha_heat,nuclide_beta_heat,nuclide_gamma_heat,nuclide_dose,nuclide_ingestion,nuclide_inhalation
0,100,3,1,Al,26,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,100,3,2,Al,26,,1.110297e+11,4.791169e-12,0.003401,0.0,0.003401,0.0,1.675088e-18,0.0,2.173910e-19,1.457697e-18,1.085307e-15,1.190471e-11,6.802694e-11
2,100,3,3,Al,26,,4.381537e+12,1.890728e-10,0.134227,0.0,0.134227,0.0,6.610360e-17,0.0,8.578851e-18,5.752474e-17,4.282920e-14,4.697929e-10,2.684531e-09
3,100,3,4,Al,26,,4.381534e+12,1.890726e-10,0.134226,0.0,0.134226,0.0,6.610355e-17,0.0,8.578846e-18,5.752471e-17,4.282917e-14,4.697926e-10,2.684529e-09
4,100,3,5,Al,26,,5.523492e+12,2.383506e-10,0.169210,0.0,0.169210,0.0,8.333209e-17,0.0,1.081475e-17,7.251735e-17,5.399172e-14,5.922345e-10,3.384197e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13970,100,3,61,Zr,97,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
13971,100,3,62,Zr,97,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
13972,100,3,63,Zr,97,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
13973,100,3,64,Zr,97,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [92]:
def save_timestep_nucludes(con, ds):
    columns = [
        "nuclide_atoms",
        "nuclide_grams",
        "nuclide_activity",
        "nuclide_alpha_activity",
        "nuclide_beta_activity",
        "nuclide_gamma_activity",
        "nuclide_heat",
        "nuclide_alpha_heat",
        "nuclide_beta_heat",
        "nuclide_gamma_heat",
        "nuclide_dose",
        "nuclide_ingestion",
        "nuclide_inhalation",
    ]
    columns_all = [
        "material_id",
        "case_id",
        "time_step_number",
        "element",
        "mass_number",
        "state",
    ] + columns

    tn = ds[columns] \
        .stack(idx=("material_id", "case_id", "time_step_number")) \
        .to_dataframe()[columns] \
        .reset_index()[columns_all] \
        .fillna(0.0)
    sql = "insert into timestep_nuclide select * from tn"
    con.execute(sql)
    con.commit()

In [93]:
save_timestep_nucludes(con, ds)

In [94]:
con.execute("select * from timestep_nuclide").df()

,material_id,case_id,time_step_number,element,mass_number,state,atoms,grams,activity,alpha_activity,beta_activity,gamma_activity,heat,alpha_heat,beta_heat,gamma_heat,dose,ingestion,inhalation
0,100,3,1,Al,26,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,100,3,2,Al,26,,1.110297e+11,4.791169e-12,0.003401,0.0,0.003401,0.0,1.675088e-18,0.0,2.173910e-19,1.457697e-18,1.085307e-15,1.190471e-11,6.802694e-11
2,100,3,3,Al,26,,4.381537e+12,1.890728e-10,0.134227,0.0,0.134227,0.0,6.610360e-17,0.0,8.578851e-18,5.752475e-17,4.282920e-14,4.697929e-10,2.684531e-09
3,100,3,4,Al,26,,4.381534e+12,1.890726e-10,0.134226,0.0,0.134226,0.0,6.610355e-17,0.0,8.578846e-18,5.752471e-17,4.282917e-14,4.697926e-10,2.684529e-09
4,100,3,5,Al,26,,5.523492e+12,2.383506e-10,0.169210,0.0,0.169210,0.0,8.333209e-17,0.0,1.081475e-17,7.251735e-17,5.399172e-14,5.922345e-10,3.384197e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13970,100,3,61,Zr,97,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
13971,100,3,62,Zr,97,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
13972,100,3,63,Zr,97,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
13973,100,3,64,Zr,97,,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [98]:
ds.gamma.to_dataframe().reset_index()[["material_id", "case_id", "time_step_number", "gamma_boundaries", "gamma"]]

,material_id,case_id,time_step_number,gamma_boundaries,gamma
0,100,3,1,1.000000e-11,0.000000e+00
1,100,3,1,1.000000e-02,3.156906e-11
2,100,3,1,2.000000e-02,0.000000e+00
3,100,3,1,5.000000e-02,0.000000e+00
4,100,3,1,1.000000e-01,5.480877e-10
...,...,...,...,...,...
1620,100,3,65,8.000000e+00,0.000000e+00
1621,100,3,65,1.000000e+01,0.000000e+00
1622,100,3,65,1.200000e+01,0.000000e+00
1623,100,3,65,1.400000e+01,0.000000e+00


In [99]:
def save_gamma_spectra(con, ds):
    columns = ["material_id", "case_id", "time_step_number", "gamma_boundaries", "gamma"]
    tg = ds.gamma.to_dataframe().reset_index()[columns]
    sql = "insert into timestep_gamma select * from tg"
    con.execute(sql)
    con.commit()

In [100]:
save_gamma_spectra(con, ds)

In [101]:
con.execute("select * from timestep_gamma").df()

,material_id,case_id,time_step_number,boundary,intensity
0,100,3,1,1.000000e-11,0.000000e+00
1,100,3,1,1.000000e-02,3.156906e-11
2,100,3,1,2.000000e-02,0.000000e+00
3,100,3,1,5.000000e-02,0.000000e+00
4,100,3,1,1.000000e-01,5.480877e-10
...,...,...,...,...,...
1620,100,3,65,8.000000e+00,0.000000e+00
1621,100,3,65,1.000000e+01,0.000000e+00
1622,100,3,65,1.200000e+01,0.000000e+00
1623,100,3,65,1.400000e+01,0.000000e+00


In [103]:
con.execute("select boundary, intensity from timestep_gamma where time_step_number = 42").df()

,boundary,intensity
0,1.000000e-11,0.000000e+00
1,1.000000e-02,4.204333e+05
2,2.000000e-02,8.981064e+04
3,5.000000e-02,1.969102e+04
4,1.000000e-01,2.891012e+06
5,2.000000e-01,7.068460e+06
6,3.000000e-01,1.825296e+06
7,4.000000e-01,1.259209e+06
8,6.000000e-01,8.268742e+06
9,8.000000e-01,9.028931e+05


In [286]:
con.close()